# Análisis Exploratorio de Datos - Sistema Fotovoltaico
## Fase 1: Carga, Limpieza y Análisis de Datos

**Objetivo:** Analizar el dataset de generación FV y desarrollar un modelo de regresión por Mínimos Cuadrados (OLS) para predecir la generación eléctrica.


In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy import stats

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("✅ Librerías importadas correctamente")


## 1. Carga y Limpieza de Datos

**Importante:** El archivo usa `;` como delimitador y `,` como separador decimal.


In [ ]:
# Cargar el dataset con los parámetros correctos
df = pd.read_csv('01 - Generacion FV.csv', sep=';', decimal=',')

print("="*80)
print("INFORMACIÓN DEL DATASET")
print("="*80)
print(f"\n📊 Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas\n")

# Mostrar información del dataset
df.info()


In [ ]:
# Mostrar primeras filas
print("\n" + "="*80)
print("PRIMERAS 10 FILAS DEL DATASET")
print("="*80)
df.head(10)


In [ ]:
# Verificar valores nulos
print("\n" + "="*80)
print("VALORES NULOS POR COLUMNA")
print("="*80)
valores_nulos = df.isnull().sum()
print(valores_nulos)
print(f"\n✅ Total de valores nulos: {valores_nulos.sum()}")


In [ ]:
# Estadísticas descriptivas
print("\n" + "="*80)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("="*80)
df.describe()


## 2. Análisis Exploratorio de Datos (EDA)

### Matriz de Correlación y Scatter Plots


In [ ]:
# Matriz de correlación
print("\n" + "="*80)
print("MATRIZ DE CORRELACIÓN")
print("="*80)

# Seleccionar solo columnas numéricas
numeric_columns = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_columns].corr()

# Mostrar correlación con generacion_W
print("\n📊 Correlación con generacion_W:")
print(correlation_matrix['generacion_W'].sort_values(ascending=False))

# Visualizar matriz de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.3f', square=True, linewidths=1)
plt.title('Matriz de Correlación - Variables del Sistema FV', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Scatter plots: generacion_W vs variables predictoras
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Relación entre Generación (W) y Variables Predictoras', 
             fontsize=16, fontweight='bold', y=1.00)

# 1. generacion_W vs irradiance_Wm2
axes[0, 0].scatter(df['irradiance_Wm2'], df['generacion_W'], alpha=0.5, s=10, color='orange')
axes[0, 0].set_xlabel('Irradiancia (W/m²)', fontsize=12)
axes[0, 0].set_ylabel('Generación (W)', fontsize=12)
axes[0, 0].set_title('Generación vs Irradiancia', fontsize=13, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Añadir línea de tendencia
z = np.polyfit(df['irradiance_Wm2'], df['generacion_W'], 1)
p = np.poly1d(z)
axes[0, 0].plot(df['irradiance_Wm2'], p(df['irradiance_Wm2']), 
                "r--", linewidth=2, label=f'Tendencia: y={z[0]:.4f}x+{z[1]:.2f}')
axes[0, 0].legend()

# 2. generacion_W vs temperatura_ambiental_°C
axes[0, 1].scatter(df['temperatura_ambiental_°C'], df['generacion_W'], 
                   alpha=0.5, s=10, color='red')
axes[0, 1].set_xlabel('Temperatura Ambiental (°C)', fontsize=12)
axes[0, 1].set_ylabel('Generación (W)', fontsize=12)
axes[0, 1].set_title('Generación vs Temperatura', fontsize=13, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Añadir línea de tendencia
z = np.polyfit(df['temperatura_ambiental_°C'], df['generacion_W'], 1)
p = np.poly1d(z)
axes[0, 1].plot(df['temperatura_ambiental_°C'], p(df['temperatura_ambiental_°C']), 
                "r--", linewidth=2, label=f'Tendencia: y={z[0]:.4f}x+{z[1]:.2f}')
axes[0, 1].legend()

# 3. generacion_W vs inclinacion_°
axes[1, 0].scatter(df['inclinacion_°'], df['generacion_W'], alpha=0.5, s=10, color='blue')
axes[1, 0].set_xlabel('Inclinación (°)', fontsize=12)
axes[1, 0].set_ylabel('Generación (W)', fontsize=12)
axes[1, 0].set_title('Generación vs Inclinación', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Boxplot: generacion_W por sky_state
df_plot = df[df['generacion_W'] > 0]  # Solo valores con generación
sns.boxplot(data=df_plot, x='sky_state', y='generacion_W', ax=axes[1, 1])
axes[1, 1].set_xlabel('Estado del Cielo', fontsize=12)
axes[1, 1].set_ylabel('Generación (W)', fontsize=12)
axes[1, 1].set_title('Distribución de Generación por Estado del Cielo', 
                     fontsize=13, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 3. Modelo de Regresión por Mínimos Cuadrados (OLS)

### Ecuación del Modelo:
$$generacion\_W = \beta_0 + \beta_1 \cdot irradiance\_Wm^2 + \beta_2 \cdot temperatura\_ambiental_{°C} + \beta_3 \cdot inclinacion_° + \epsilon$$


In [ ]:
# Filtrar datos con generación > 0 para el modelo
df_modelo = df[df['generacion_W'] > 0].copy()

print("="*80)
print("DATOS PARA MODELADO")
print("="*80)
print(f"📊 Datos originales: {len(df)} filas")
print(f"📊 Datos con generación > 0: {len(df_modelo)} filas")
print(f"📊 Porcentaje útil: {len(df_modelo)/len(df)*100:.2f}%")


In [ ]:
# Preparar variables para el modelo
# Variables predictoras (X)
X = df_modelo[['irradiance_Wm2', 'temperatura_ambiental_°C', 'inclinacion_°']]

# Variable objetivo (y)
y = df_modelo['generacion_W']

# Añadir constante (intercepto)
X_with_const = sm.add_constant(X)

print("="*80)
print("VARIABLES DEL MODELO")
print("="*80)
print(f"\n📌 Variables predictoras (X): {list(X.columns)}")
print(f"📌 Variable objetivo (y): generacion_W")
print(f"📌 Número de observaciones: {len(X)}")


In [ ]:
# Ajustar el modelo OLS
modelo_ols = sm.OLS(y, X_with_const).fit()

# Mostrar resumen completo del modelo
print("="*80)
print("RESUMEN DEL MODELO OLS")
print("="*80)
print(modelo_ols.summary())


In [ ]:
# Extraer y mostrar coeficientes de manera clara
print("\n" + "="*80)
print("COEFICIENTES DEL MODELO")
print("="*80)

coeficientes = pd.DataFrame({
    'Variable': ['Intercepto (β₀)', 'Irradiancia (β₁)', 'Temperatura (β₂)', 'Inclinación (β₃)'],
    'Coeficiente': modelo_ols.params.values,
    'Error Estándar': modelo_ols.bse.values,
    'p-value': modelo_ols.pvalues.values,
    'IC 95% Inferior': modelo_ols.conf_int()[0].values,
    'IC 95% Superior': modelo_ols.conf_int()[1].values
})

print(coeficientes.to_string(index=False))

print("\n" + "="*80)
print("ECUACIÓN FINAL DEL MODELO")
print("="*80)
print(f"\ngeneracion_W = {modelo_ols.params[0]:.6f}")
print(f"               + {modelo_ols.params[1]:.6f} × irradiance_Wm2")
print(f"               + {modelo_ols.params[2]:.6f} × temperatura_ambiental_°C")
print(f"               + {modelo_ols.params[3]:.6f} × inclinacion_°")


In [ ]:
# Métricas de rendimiento del modelo
print("\n" + "="*80)
print("MÉTRICAS DE RENDIMIENTO")
print("="*80)

# Predicciones
y_pred = modelo_ols.predict(X_with_const)

# Calcular métricas
r2 = modelo_ols.rsquared
r2_adj = modelo_ols.rsquared_adj
rmse = np.sqrt(mean_squared_error(y, y_pred))
mae = mean_absolute_error(y, y_pred)
mape = np.mean(np.abs((y - y_pred) / y)) * 100

print(f"\n📊 R² (Coeficiente de Determinación): {r2:.6f}")
print(f"📊 R² Ajustado: {r2_adj:.6f}")
print(f"📊 RMSE (Error Cuadrático Medio): {rmse:.4f} W")
print(f"📊 MAE (Error Absoluto Medio): {mae:.4f} W")
print(f"📊 MAPE (Error Porcentual Absoluto Medio): {mape:.2f}%")
print(f"📊 AIC (Criterio de Información de Akaike): {modelo_ols.aic:.2f}")
print(f"📊 BIC (Criterio de Información Bayesiano): {modelo_ols.bic:.2f}")

print(f"\n✅ El modelo explica el {r2*100:.2f}% de la variabilidad en la generación")
print(f"✅ Error típico de predicción: ±{rmse:.2f} W")


## 4. Diagnóstico del Modelo

Verificaremos los supuestos del modelo de regresión lineal.


In [ ]:
# Calcular residuos
residuos = y - y_pred

# Gráficos de diagnóstico
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Diagnóstico del Modelo OLS', fontsize=16, fontweight='bold')

# 1. Valores Reales vs Predichos
axes[0, 0].scatter(y, y_pred, alpha=0.5, s=10)
axes[0, 0].plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Valores Reales (W)', fontsize=11)
axes[0, 0].set_ylabel('Valores Predichos (W)', fontsize=11)
axes[0, 0].set_title('Valores Reales vs Predichos', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 2. Residuos vs Valores Predichos
axes[0, 1].scatter(y_pred, residuos, alpha=0.5, s=10)
axes[0, 1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[0, 1].set_xlabel('Valores Predichos (W)', fontsize=11)
axes[0, 1].set_ylabel('Residuos (W)', fontsize=11)
axes[0, 1].set_title('Residuos vs Valores Predichos', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 3. Histograma de Residuos
axes[1, 0].hist(residuos, bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].axvline(x=0, color='r', linestyle='--', lw=2)
axes[1, 0].set_xlabel('Residuos (W)', fontsize=11)
axes[1, 0].set_ylabel('Frecuencia', fontsize=11)
axes[1, 0].set_title('Distribución de Residuos', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Q-Q Plot (Normalidad de Residuos)
stats.probplot(residuos, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot (Normalidad de Residuos)', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. Exportar Coeficientes del Modelo

Guardaremos los coeficientes para usarlos en el simulador web.


In [ ]:
import json

# Preparar diccionario con los parámetros del modelo
modelo_params = {
    'coeficientes': {
        'intercepto': float(modelo_ols.params[0]),
        'irradiance_Wm2': float(modelo_ols.params[1]),
        'temperatura_ambiental_C': float(modelo_ols.params[2]),
        'inclinacion': float(modelo_ols.params[3])
    },
    'metricas': {
        'r2': float(modelo_ols.rsquared),
        'r2_ajustado': float(modelo_ols.rsquared_adj),
        'rmse': float(rmse),
        'mae': float(mae),
        'mape': float(mape)
    },
    'ecuacion': f"generacion_W = {modelo_ols.params[0]:.6f} + {modelo_ols.params[1]:.6f}*I + {modelo_ols.params[2]:.6f}*T + {modelo_ols.params[3]:.6f}*θ"
}

# Guardar en archivo JSON
with open('modelo_coeficientes.json', 'w', encoding='utf-8') as f:
    json.dump(modelo_params, f, indent=4, ensure_ascii=False)

print("="*80)
print("COEFICIENTES EXPORTADOS")
print("="*80)
print("\n✅ Coeficientes guardados en 'modelo_coeficientes.json'")
print("\n📄 Contenido del archivo:")
print(json.dumps(modelo_params, indent=4, ensure_ascii=False))
